In [ ]:
pip install pennylane

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cudaq

kernel = cudaq.make_kernel()

In [ ]:
import pennylane as qml
import pennylane.numpy as np

# Define the Molecule
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])   # Adjust coordinates based on bond length
electrons = 2  # Number of active electrons

# Define the number of qubits and the Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("Hamiltonian:\n", H)

# Define the total spin operator squared
S2 = qml.qchem.spin2(electrons, qubits)

# Generate the Hartree-Fock state
hf = qml.qchem.hf_state(electrons, qubits)



# Generate single and double excitations for the ground state
singles_ground, doubles_ground = qml.qchem.excitations(electrons, qubits, delta_sz=0)

# Generate single and double excitations for the excited state (S = 1)
singles_excited, doubles_excited = qml.qchem.excitations(electrons, qubits, delta_sz=1)

# Define the circuit to apply single and double excitations to the Hartree-Fock state
def circuit(params, wires, singles, doubles):
    qml.AllSinglesDoubles(params, wires, hf, singles, doubles)

# Define the quantum device
dev = qml.device("lightning.qubit", wires=qubits)

# Define the cost function to compute the expectation value of the Hamiltonian
@qml.qnode(dev, interface="autograd")
def cost_fn(params, singles, doubles):
    circuit(params, wires=range(qubits), singles=singles, doubles=doubles)
    return qml.expval(H)

# Define the function to compute the expectation value of the spin operator
@qml.qnode(dev, interface="autograd")
def S2_exp_value(params, singles, doubles):
    circuit(params, wires=range(qubits), singles=singles, doubles=doubles)
    return qml.expval(S2)

# Define the function to compute the total spin
def total_spin(params, singles, doubles):
    return -0.5 + np.sqrt(1 / 4 + S2_exp_value(params, singles, doubles))

# Define the optimization loop for ground states
def optimize_state(singles, doubles, max_iterations=500, conv_tol=1e-6, optimizer="Adam", stepsize=0.01):
    if optimizer == "Adam":
        opt = qml.AdamOptimizer(stepsize)
    elif optimizer == "RMSProp":
        opt = qml.RMSPropOptimizer(stepsize)
    else:
        opt = qml.GradientDescentOptimizer(stepsize)

    np.random.seed(0)  # for reproducibility
    theta = np.random.normal(0, np.pi, len(singles) + len(doubles), requires_grad=True)

    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(lambda params: cost_fn(params, singles, doubles), theta)

        energy = cost_fn(theta, singles, doubles)
        spin = total_spin(theta, singles, doubles)

        conv = np.abs(energy - prev_energy)
        if n % 10 == 0:
            print(f"Step = {n}, Energy = {energy:.8f} Ha, S = {spin:.4f}")

        if conv <= conv_tol:
            break

    return theta, energy, spin

# Optimize the ground state using different optimizer and adjustments
print("\nOptimizing the ground state with Adam optimizer:")
theta_ground_adam, energy_ground_adam, spin_ground_adam = optimize_state(singles_ground, doubles_ground, optimizer="Adam", stepsize=0.01)
print("\nFinal value of the ground-state energy (Adam optimizer):", energy_ground_adam)
print("Optimal value of the circuit parameters:", theta_ground_adam)

# Optimize the ground state using different optimizer and adjustments
print("\nOptimizing the ground state with RMSProp optimizer:")
theta_ground_rmsprop, energy_ground_rmsprop, spin_ground_rmsprop = optimize_state(singles_ground, doubles_ground, optimizer="RMSProp", stepsize=0.01)
print("\nFinal value of the ground-state energy (RMSProp optimizer):", energy_ground_rmsprop)
print("Optimal value of the circuit parameters:", theta_ground_rmsprop)

"""# Finding the lowest-lying excited state with  S = 1"""

# Generate single and double excitations for the excited state (S = 1)
singles_excited, doubles_excited = qml.qchem.excitations(electrons, qubits, delta_sz=1)

# Define the circuit to apply single and double excitations to the Hartree-Fock state
def circuit(params, wires, singles, doubles):
    qml.AllSinglesDoubles(params, wires, hf, singles, doubles)

# Define the quantum device
dev = qml.device("lightning.qubit", wires=qubits)

# Define the cost function to compute the expectation value of the Hamiltonian
@qml.qnode(dev, interface="autograd")
def cost_fn(params, singles, doubles):
    circuit(params, wires=range(qubits), singles=singles_excited, doubles=doubles_excited)
    return qml.expval(H)

# Define the function to compute the expectation value of the spin operator
@qml.qnode(dev, interface="autograd")
def S2_exp_value(params, singles, doubles):
    circuit(params, wires=range(qubits), singles=singles_excited, doubles=doubles_excited)
    return qml.expval(S2)

# Define the function to compute the total spin
def total_spin(params, singles_excited, doubles_excited):
    return -0.5 + np.sqrt(1 / 4 + S2_exp_value(params, singles_excited, doubles_excited))

"""Looping again"""

# Define the optimization loop for ground states
def optimize_state(singles_excited, doubles_excited, max_iterations=500, conv_tol=1e-6, optimizer="Adam", stepsize=0.01):
    if optimizer == "Adam":
        opt = qml.AdamOptimizer(stepsize)
    elif optimizer == "RMSProp":
        opt = qml.RMSPropOptimizer(stepsize)
    else:
        opt = qml.GradientDescentOptimizer(stepsize)

    np.random.seed(0)  # for reproducibility
    theta = np.random.normal(0, np.pi, len(singles_excited) + len(doubles_excited), requires_grad=True)

    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(lambda params: cost_fn(params, singles_excited, doubles_excited), theta)

        energy = cost_fn(theta, singles_excited, doubles_excited)
        spin = total_spin(theta, singles_excited, doubles_excited)

        conv = np.abs(energy - prev_energy)
        if n % 10 == 0:
            print(f"Step = {n}, Energy = {energy:.8f} Ha, S = {spin:.4f}")

        if conv <= conv_tol:
            break

    return theta, energy, spin

# Optimize the ground state using different optimiser and adjustments
print("\nOptimizing the Excited state with Adam optimizer:")
theta_excited_adam, energy_excited_adam, spin_excited_adam = optimize_state(singles_excited, doubles_excited, optimizer="Adam", stepsize=0.01)
print("\nFinal value of the excited-state energy (Adam optimizer):", energy_excited_adam)
print("Optimal value of the circuit parameters:", theta_excited_adam)

# Optimize the ground state using different optimiser and adjustments
print("\nOptimizing the Excited state with RMSProp optimizer:")
theta_excited_rmsprop, energy_excited_rmsprop, spin_excited_rmsprop = optimize_state(singles_excited, doubles_excited, optimizer="RMSProp", stepsize=0.01)
print("\nFinal value of the excited-state energy (RMSProp optimizer):", energy_excited_rmsprop)
print("Optimal value of the circuit parameters:", theta_excited_rmsprop)


# Calculate spectral gap
spectral_gap_adam = energy_excited_adam - energy_ground_adam
print("\nSpectral Gap (Adam optimizer):", spectral_gap_adam)

spectral_gap_rmsprop = energy_excited_rmsprop - energy_ground_rmsprop
print("Spectral Gap (RMSProp optimizer):", spectral_gap_rmsprop)

Number of qubits =  4
Hamiltonian:
   (-0.2427450126094144) [Z2]
+ (-0.2427450126094144) [Z3]
+ (-0.042072551947439224) [I0]
+ (0.1777135822909176) [Z0]
+ (0.1777135822909176) [Z1]
+ (0.12293330449299361) [Z0 Z2]
+ (0.12293330449299361) [Z1 Z3]
+ (0.16768338855601356) [Z0 Z3]
+ (0.16768338855601356) [Z1 Z2]
+ (0.17059759276836803) [Z0 Z1]
+ (0.1762766139418181) [Z2 Z3]
+ (-0.044750084063019925) [Y0 Y1 X2 X3]
+ (-0.044750084063019925) [X0 X1 Y2 Y3]
+ (0.044750084063019925) [Y0 X1 X2 Y3]
+ (0.044750084063019925) [X0 Y1 Y2 X3]

Optimizing the ground state with Adam optimizer:
Step = 0, Energy = 0.13315273 Ha, S = 0.0657
Step = 10, Energy = 0.03985859 Ha, S = 0.0714
Step = 20, Energy = -0.05816564 Ha, S = 0.0766
Step = 30, Energy = -0.15721644 Ha, S = 0.0812
Step = 40, Energy = -0.25271217 Ha, S = 0.0847
Step = 50, Energy = -0.34017730 Ha, S = 0.0869
Step = 60, Energy = -0.41646766 Ha, S = 0.0875
Step = 70, Energy = -0.48081015 Ha, S = 0.0867
Step = 80, Energy = -0.53530276 Ha, S = 0.0842
